In [24]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler,OneHotEncoder,LabelEncoder
from sklearn.model_selection import train_test_split
import pickle

In [2]:
data=pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
data=data.drop(["RowNumber","CustomerId","Surname"],axis=1)
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [9]:
label_encoder=LabelEncoder()
label_encoder_gender=label_encoder.fit_transform(data["Gender"])
label_encoder_gender

array([0, 0, 0, ..., 0, 1, 0], shape=(10000,))

In [28]:
one_hot_encoder=OneHotEncoder()
data['Gender']=one_hot_encoder.fit_transform(data[["Geography"]]).toarray()
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,1.0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0.0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,1.0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,1.0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0.0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1.0,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1.0,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,1.0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,0.0,42,3,75075.31,2,1,0,92888.52,1


In [29]:
geo_encoded_df=pd.DataFrame(geo_encoded,columns=one_hot_encoder.get_feature_names_out(["Geography"]))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [30]:
df=pd.concat([data.drop("Geography",axis=1),geo_encoded_df],axis=1)
df.head()


,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,1.0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0.0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,1.0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,1.0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0.0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [31]:

X=df.drop('EstimatedSalary',axis=1)
y=df['EstimatedSalary']

In [32]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [34]:
with open('label_encoder.pkl','wb') as file:
    pickle.dump(label_encoder,file)
    
with open('one_hot_encoder.pkl','wb') as file:
    pickle.dump(one_hot_encoder,file)
    
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

In [35]:
#ANN Regression Problem statement
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

model=Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)),
    Dense(32,activation='relu'),
    Dense(1)
])

c:\Users\Suhail.DESKTOP-0CIIIA7\OneDrive\Desktop\New folder\ann\.venv\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [36]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [39]:
model.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mae'])

In [40]:
#Setup the tensorboard
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard

log_dir="regressionlogs/fit/"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [41]:
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [42]:
history=model.fit(
    X_train,y_train,
    validation_data=(X_test,y_test),
    epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - loss: 100363.0234 - mae: 100363.0234 - val_loss: 98455.5234 - val_mae: 98455.5234
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 99384.0000 - mae: 99384.0000 - val_loss: 96460.9375 - val_mae: 96460.9375
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 95933.3281 - mae: 95933.3281 - val_loss: 91396.6562 - val_mae: 91396.6562
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 89213.4609 - mae: 89213.4609 - val_loss: 83151.8516 - val_mae: 83151.8516
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 79769.3516 - mae: 79769.3516 - val_loss: 73097.8984 - val_mae: 73097.8984
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 69488.6562 - mae: 69488.6562 - val_loss: 63692.6602 - val_mae: 63692.6602
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 60696.7227 - mae: 60696.7227 - val_loss: 56649.6289 - val_mae: 56649.6289
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step 

In [43]:
model.save('regression.h5')

In [44]:
%load_ext tensorboard

In [46]:
%tensorboard --logdir regressionlogs/fit

Reusing TensorBoard on port 6006 (pid 2100), started 0:01:45 ago. (Use '!kill 2100' to kill it.)